In [1]:
import pandas as pd
import re

df_low_freq_words = pd.read_csv("all_words_freq_sender.csv")
df_low_freq_words.head(2)

,word_processed,freq
0,a,23
1,ab,2


In [2]:
df_low_freq_words = df_low_freq_words.sort_values(["freq"], ascending=[False])
df_low_freq_words.tail(5)

,word_processed,freq
54,best,2
269,number,2
270,nuys,2
271,nvr,2
216,letter,2


In [3]:
low_weightage_words = list(df_low_freq_words[df_low_freq_words["freq"]>3]["word_processed"])
low_weightage_words = list(set(low_weightage_words))

In [4]:
def word2ngrams(text, n=3):
  """ Convert word into character ngrams. """
  return set([text[i:i+n] for i in range(len(text)-n+1)])

In [6]:
final_low_freq_words = []
#remove non-alpha characters
for i,word in enumerate(low_weightage_words):
    if not pd.isna(word):
        word = re.sub('[^0-9a-zA-Z]+', '', word)
        # word = re.sub(' +', ' ', word)
        final_low_freq_words.append(word)
        

In [7]:
## for each i-gram of both gt and pred, get max(recall, precision).. and then average it out 
## (currently given equal weightage to all i-grams)

def get_functional_metrics(gt, pred):
    #remove non-alpha characters
    gt = re.sub('[^0-9a-zA-Z]+', ' ', gt)
    pred = re.sub('[^0-9a-zA-Z]+', ' ', pred)
    
    #remove extra spaces in bw words
    gt = re.sub(' +', ' ', gt)
    pred = re.sub(' +', ' ', pred)
    
    if gt=='' or pred=='':
        return [0, pred]
    
    ## remove low-weighted words to get normalized strings
    gt_words = gt.lower().split(' ')
    pred_words = pred.lower().split(' ')

    norm_gt_words = []
    for word in gt_words:
        if word not in final_low_freq_words:
            norm_gt_words.append(word)
    norm_gt = ''.join(norm_gt_words)  

    norm_pred_words = []
    for word in pred_words:
        if word not in final_low_freq_words:
            norm_pred_words.append(word)
    norm_pred = ''.join(norm_pred_words)

    print("norm gt, norm pred : ", norm_gt, norm_pred)

    len_norm_gt, len_norm_pred = len(norm_gt), len(norm_pred)
    
    if len_norm_gt==0 and len_norm_pred==0:
        return [1, norm_pred]
    
    if len_norm_gt==0 or len_norm_pred==0:
        return [0, norm_pred]
    
    max_ngrams = int((min(len_norm_gt,len_norm_pred)+1)/2)
    
    if max_ngrams==0:
        return [0, norm_pred]
    
    avg_metric=0
    
    if len_norm_gt<3 or len_norm_pred<3:
        if norm_gt==norm_pred:
            return [1, norm_pred]
        else:
            return [0, norm_pred]
        
    for i in range(2, max_ngrams+1):
        gt_igrams = word2ngrams(norm_gt, n=i)
        pred_igrams = word2ngrams(norm_pred, n=i)

        n_common_elements = len(gt_igrams) - (len(gt_igrams.difference(pred_igrams)))
        # print("n_common_elements : ", n_common_elements)
        precision = n_common_elements/len(pred_igrams)
        recall = n_common_elements/len(gt_igrams)
        avg_metric += (max(precision, recall))

    return [avg_metric/(max_ngrams-1), norm_pred]

In [8]:
df1 = pd.read_csv("~/Downloads/gl_sender_jan24.csv")
df1.columns

Index(['customer_id', 'customer_name', 'queue_name', 'doc_id',
       'extracted_value', 'extraction_confidence', 'validated_value',
       'direct_match', 'fuzzy_match', 'fuzzy_score', 'high_confidence',
       'document_class'],
      dtype='object')

In [9]:
df1 = df1.fillna('')

In [10]:
df1[['func_metrics_score', 'norm_pred']] = df1.apply(lambda x: pd.Series(get_functional_metrics(x['validated_value'], x['extracted_value'])), axis=1)

gt pred :  ['ben', 'hyatt'] ['ben', 'hyatt']
norm gt, norm pred :  benhyatt benhyatt
i weighted_f1_score_num :  2 1.0
i weighted_f1_score_num :  3 2.0
i weighted_f1_score_num :  4 3.0
gt pred :  ['palmetto', 'investment', 'group', 'inc', ''] ['general', 'release', 'of', 'all', 'claims']
norm gt, norm pred :  palmettoinvestment generalreleaseall
i weighted_f1_score_num :  2 0.13333333333333333
[DEBUG] : exception :  float division by zero
i weighted_f1_score_num :  3 0
[DEBUG] : exception :  float division by zero
i weighted_f1_score_num :  4 0
[DEBUG] : exception :  float division by zero
i weighted_f1_score_num :  5 0
[DEBUG] : exception :  float division by zero
i weighted_f1_score_num :  6 0
[DEBUG] : exception :  float division by zero
i weighted_f1_score_num :  7 0
[DEBUG] : exception :  float division by zero
i weighted_f1_score_num :  8 0
[DEBUG] : exception :  float division by zero
i weighted_f1_score_num :  9 0
gt pred :  ['auto', 'owners', 'insurance'] ['palmetto', 'investme

## v0 -> weighted average of f1-score of each i-gram

In [1]:
def get_functional_metrics(gt,pred):

    #remove extra spaces in bw words
    gt = re.sub(' +', ' ', gt)
    pred = re.sub(' +', ' ', pred)
    
    ## remove low- weightaged words from both GT and pred
    gt_words = gt.lower().split(' ')
    pred_words = pred.lower().split(' ')

    norm_gt_words = []
    for word in gt_words:
        if word not in low_weightage_words:
            norm_gt_words.append(word)
    norm_gt = ' '.join(norm_gt_words)  

    norm_pred_words = []
    for word in pred_words:
        if word not in low_weightage_words:
            norm_pred_words.append(word)
    norm_pred = ' '.join(norm_pred_words)

    print("norm gt, norm pred : ", norm_gt,"\n", norm_pred)

    # get n-grams for both GT and Pred, and find the precision and recall bw them
    len_norm_gt, len_norm_pred = len(norm_gt), len(norm_pred)
    max_ngrams = (min(len_norm_gt,len_norm_pred)) # -1/2 ??
    weighted_f1_score_num = 0

    for i in range(2, max_ngrams+1):
        gt_igrams = word2ngrams(norm_gt, n=i)
        pred_igrams = word2ngrams(norm_pred, n=i)

        n_common_elements = len(gt_igrams) - (len(gt_igrams.difference(pred_igrams)))
        # print("n_common_elements : ", n_common_elements)
        precision = n_common_elements/len(pred_igrams)
        recall = n_common_elements/len(gt_igrams)

        # print(f"precision and recall at {i} is {precision} and {recall}")
        i_f1_score = 2*precision*recall / (precision+recall)
        weighted_f1_score_num += ((i/max_ngrams)*i_f1_score)
        # print("i ((i/max_ngrams)*fp) : ", i, ((i/max_ngrams)*i_f1_score), i_f1_score)

    weighted_f1_score_denom = (max_ngrams+1)/2 - (1/max_ngrams)
    print("weighted_f1_score_denom : ", weighted_f1_score_denom)
    return weighted_f1_score_num/weighted_f1_score_denom


        
